In [5]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import json
import pandas as pd
import networkx as nx
import random
import matplotlib.pyplot as plt
import numpy as np
import math
import pickle
from scipy.spatial import Voronoi, voronoi_plot_2d


from shapely.geometry import LineString, Polygon, Point, MultiLineString, MultiPolygon

import numpy as np
from stl import mesh
import trimesh
from shapely.ops import unary_union
from shapely.validation import make_valid


from shapely.geometry import LineString, Polygon, Point, MultiLineString
import numpy as np


Functions to generate STL files

In [4]:
# Function to convert 2D polygon to 3D mesh by adding thickness
def polygon_to_3d_mesh(polygon, thickness=1.0):
    # Get the exterior coordinates of the 2D polygon
    exterior_coords = list(polygon.exterior.coords)
    
    # Create vertices for the top and bottom of the 3D shape
    top_vertices = [(x, y, thickness) for x, y in exterior_coords]
    bottom_vertices = [(x, y, 0) for x, y in exterior_coords]
    
    # Vertices array: two sets of vertices (top and bottom)
    vertices = np.array(top_vertices + bottom_vertices)
    
    # Number of points in the polygon
    n = len(exterior_coords)
    
    # Create faces (triangles) connecting the top and bottom surfaces
    faces = []
    
    # Create side walls
    for i in range(n):
        next_i = (i + 1) % n
        faces.append([i, next_i, n + next_i])   # Top to bottom
        faces.append([i, n + next_i, n + i])    # Bottom to top
    
    # Create top and bottom surfaces
    for i in range(1, n - 1):
        faces.append([0, i+1, i ])#[0, i, i + 1])             # Top face
        faces.append([n, n + i, n + i+1]) #[n, n + i, n + i + 1])     # Bottom face

    # Convert faces to NumPy array
    faces = np.array(faces)
    
    # Create mesh object
    polygon_mesh = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
    
    for i, face in enumerate(faces):
        for j in range(3):
            polygon_mesh.vectors[i][j] = vertices[face[j], :]
    
    return polygon_mesh

def merge_3d_meshes(mesh_list):
    # List to hold the vertices and faces of the merged mesh
    vertices = []
    faces = []
    
    # Variable to track the current offset for the face indices
    vertex_offset = 0
    
    # Iterate over each mesh and extract its vertices and faces
    for m in mesh_list:
        # Extract the vertices and faces of the current mesh
        current_vertices = m.vectors.reshape(-1, 3)  # Each face is a set of 3 vertices
        current_faces = np.arange(len(current_vertices)).reshape(-1, 3)
        
        # Append the vertices, and adjust the face indices by the current offset
        vertices.append(current_vertices)
        faces.append(current_faces + vertex_offset)
        
        # Update the vertex offset for the next mesh
        vertex_offset += len(current_vertices)
    
    # Concatenate all the vertices and faces into a single array
    vertices = np.vstack(vertices)
    faces = np.vstack(faces)
    
    # Create a new merged mesh
    merged_mesh = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
    
    # Fill the new mesh with the vertices and faces
    for i, face in enumerate(faces):
        for j in range(3):
            merged_mesh.vectors[i][j] = vertices[face[j], :]
    
    return merged_mesh

def save_stl(poly_dict, out_plane_thickness, name, f_x=1, f_y = 1, frame_thickness = None):
    mesh_list = []
    for k,v in poly_dict.items():
        p = [] 
        for j in v:
            try: 
                p.append((float(j[0]), float(j[1])))
            except:
                None
        
        mesh_list.append(polygon_to_3d_mesh(Polygon(p), thickness=out_plane_thickness))
    
    if frame_thickness != None:
        t = frame_thickness
        bottom = Polygon([ (0,0-t), (0,0),(f_x,0),(f_x,0-t)])
        top = Polygon([(0,f_y),(0,f_y+t), (f_x,f_y+t), (f_x,f_y)])
        #right = Polygon([(f_x,0-t), (f_x,f_y+t), (f_x+t,f_y+t), (f_x+t,0-t)])
        #left = Polygon([(0-t,0-t),(0-t,f_y+t), (0,f_y+t), (0,0-t)])

        f = [bottom,top]#,  right, left]

        for f_ in f:            
            mesh_list.append(polygon_to_3d_mesh(f_, thickness=out_plane_thickness))

    merged_mesh = merge_3d_meshes(mesh_list)

    # Save the merged mesh as an STL file   
    merged_mesh.save(name)

Voronoi tessellation

In [6]:
def generate_nucleation_points(node_number, epsilon, domain_size=(1, 1)):
    """
    Generate nucleation points with minimum distance epsilon between each point.
    """
    
    width, height = domain_size
    # Start with a single random point
    nucleation_points = [(random.uniform(0, width), random.uniform(0, height))]
    
    # Try to add more points
    attempts = 0
    max_attempts = node_number * 100  # Limit attempts to avoid infinite loops
    
    while len(nucleation_points) < node_number and attempts < max_attempts:
        # Generate a random candidate point
        candidate = (random.uniform(0, width), random.uniform(0, height))
        
        # Check distance against all existing points
        valid_point = True
        for point in nucleation_points:
            distance = np.linalg.norm(np.array(candidate) - np.array(point))
            if distance < epsilon:
                valid_point = False
                break
        
        if valid_point:
            nucleation_points.append(candidate)
        
        attempts += 1
    
    print(f"Generated {len(nucleation_points)} points out of {node_number} requested")
    print(f"Attempts made: {attempts}")
    
    return nucleation_points

def voronoi_get_edges_nodes(vor):
    """
    Extract edges positions.
    """
    edge_positions = []
    
    # Plot ridges (edges)
    center = vor.points.mean(axis=0)
    ptp_bound = vor.points.ptp(axis=0)

    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        if v1 >= 0 and v2 >= 0:
            
            edge_positions.append((vor.vertices[v1], vor.vertices[v2]))

        else:
            # Infinite ridge
            i = v1 if v1 >= 0 else v2
            t = vor.points[p2] - vor.points[p1]  # tangent
            t = t / np.linalg.norm(t)
            n = np.array([-t[1], t[0]])  # normal (perpendicular)

            midpoint = vor.points[[p1, p2]].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, n)) * n
            far_point = vor.vertices[i] + direction * ptp_bound.max()
            #far_point = vor.vertices[i] - n * ptp_bound.max()

            if v1 >= 0:
                edge_positions.append((vor.vertices[v1], far_point))
            else:
                edge_positions.append((far_point, vor.vertices[v2]))
    
    return edge_positions

def trim_edges_to_bounding_box(edge_positions, bounding_box):
    """
    Trims line segments to a bounding box.
    Parameters:
        edge_positions: list of (point1, point2) tuples, where each point is a NumPy array or list.
        bounding_box: a Shapely Polygon representing the clipping boundary.
    Returns:
        List of (point1, point2) tuples that are inside or trimmed to fit the bounding box.
    """
    trimmed_edges = []

    for e in edge_positions:
        v1 = Point(e[0])
        v2 = Point(e[1])
        line = LineString([v1, v2])

        if line.within(bounding_box):
            # Line is fully inside
            trimmed_edges.append((e[0], e[1]))
        else:
            intersection = line.intersection(bounding_box)

            if intersection.is_empty:
                continue  # Line is completely outside

            if isinstance(intersection, LineString):
                coords = list(intersection.coords)
                if len(coords) >= 2:
                    trimmed_edges.append((np.array(coords[0]), np.array(coords[1])))

            elif isinstance(intersection, MultiLineString):
                for part in intersection.geoms:
                    coords = list(part.coords)
                    if len(coords) >= 2:
                        trimmed_edges.append((np.array(coords[0]), np.array(coords[1])))

    return trimmed_edges

def get_network_edges_nodes(edge_positions):
     # Create a list of unique nodes from the new edge positions
    node_list = []


    for i in edge_positions:
        v1_ = list(i[0])
        v2_ = list(i[1])
        node_list.append((v1_[0], v1_[1]))
        node_list.append((v2_[0], v2_[1]))

    node_list = list(set(node_list))

    node_map = {i:en for en, i in enumerate(node_list)}
    node_map_rev = {en: i for en, i in enumerate(node_list)}

    edge_list = [ (node_map[tuple(list(i[0]))], node_map[tuple(list(i[1]))] ) for i in edge_positions]


    return edge_list, node_map_rev


def thicken_edge(p1, p2, width, bounding_box = None):
    """Create a polygon band around a line segment from p1 to p2."""
    p1 = np.array(p1)
    p2 = np.array(p2)
    direction = p2 - p1
    direction = direction / np.linalg.norm(direction)  # Normalize

    # Get normal vector
    normal = np.array([-direction[1], direction[0]])

    # Offset points
    p1_left = p1 + (width / 2) * normal
    p1_right = p1 - (width / 2) * normal
    p2_left = p2 + (width / 2) * normal
    p2_right = p2 - (width / 2) * normal

    # Build the polygon
    band = Polygon([p1_left, p2_left, p2_right, p1_right])

    if bounding_box is not None:
        band = band.intersection(bounding_box)

    return band



In [7]:
thickness_area = [(0.005, 20), (0.0075, 30), (0.01, 40), (0.0125,50), (0.015, 60),(0.0175,70)]
sample = 5
epsilon = 3.5e-2  # Minimum distance between points  (3.5e-2 for 500)
node_number = 500 

for s in [1,2,3,4,5]:
    # Generate nucleation points
    nucleation_points = generate_nucleation_points(node_number, epsilon)
    # Create Voronoi diagram
    vor = Voronoi(nucleation_points)
    # Get edges and nodes
    edge_positions = voronoi_get_edges_nodes(vor)

    # trim edges
    bounding_box = Polygon([(0, 0), (1, 0), (1, 1), (0, 1)])
    delta = .01
    bounding_box_bigger = Polygon([(0-delta, 0-delta), (1+delta, 0-delta), (1+delta, 1+delta), (0-delta, 1+delta)])
    
    # boundary_lines is just illustrative — actual Voronoi edges go into `edge_positions`
    trimmed_edges = trim_edges_to_bounding_box(edge_positions, bounding_box_bigger)

    # Get network edges and nodes
    edge_list, node_map_rev = get_network_edges_nodes(trimmed_edges)
    
    
    for ta in thickness_area:
    
        thickness= ta[0]    
        polygon_dict = {}
        area = 0

        for en, e in enumerate(trimmed_edges):  # Assuming your trimmed Voronoi edges
            poly = thicken_edge(e[0], e[1], thickness, bounding_box)
            poly_coord = poly.exterior.coords[:]
            if len(poly_coord) > 0:
                polygon_dict[en] = poly_coord

            area += poly.area

        print(f"Total area of bands: {area}")

        thickness_string = str(round(thickness,4)).replace(r'.', r'p')
        file_name = 'sample_voronoi_r0'+'_l'+str(thickness_string)+'_v'+str(s)+'_d'+str(int(area*100)) 

           
        path = r"./"
        save_stl(polygon_dict, out_plane_thickness = .25, name = path+file_name+'.stl', frame_thickness=0.05)

        #save the polygon_dict to a pickle file
        with open(path+'input_'+file_name + '.pkl', 'wb') as f:
            pickle.dump(polygon_dict, f, protocol=2)

        #save the structure
        output = {'nucleation_points': nucleation_points, 'edge_positions': edge_positions, 'edges': edge_list, 'nodes': node_map_rev,'polygon_dict':polygon_dict ,'thickness': thickness, 'area': area}

        with open(path+file_name + '.pkl', 'wb') as f:
            pickle.dump(output, f, protocol=2)

Generated 500 points out of 500 requested
Attempts made: 9476
Total area of bands: 0.20791246502629926
Total area of bands: 0.31186741493443393
Total area of bands: 0.4158207204003348
Total area of bands: 0.5197724720607774
Total area of bands: 0.6237206134694576
Total area of bands: 0.7276601621478983
Generated 500 points out of 500 requested
Attempts made: 7201
Total area of bands: 0.208147600471974
Total area of bands: 0.31221454018045885
Total area of bands: 0.4162714409621582
Total area of bands: 0.5203116847475002
Total area of bands: 0.6243325635287871
Total area of bands: 0.7283309980019099
Generated 500 points out of 500 requested
Attempts made: 8461
Total area of bands: 0.20707548669374237
Total area of bands: 0.3106131504854152
Total area of bands: 0.4141492390470974
Total area of bands: 0.5176801493933938
Total area of bands: 0.6212036252455396
Total area of bands: 0.7247186533325053
Generated 500 points out of 500 requested
Attempts made: 9117
Total area of bands: 0.208238